# week12 과제

다음 두 가지 중에 하나를 선택하세요.
- 제공된 소스 코드의 빈칸 채우기(10점)
- 제공된 소스 코드를 수정하여 성능을 개선한 모델 만들기 (최대 15점: 10점 + 가산점 5점)
    - ipynb 파일에 성능 개선 아이디어를 서술하고, 실행 결과를 제시


hidden_size를 150으로 늘리고, layer를 2로 늘림

### layer=1 -> layer=2
### hidden_size=100 -> hidden_size=150


층과 히든 노드 개수를 늘리면 학습을 더욱 정교하게 할 수 있을 것으로 예상하고, layer 수를 2로, hidden_size를 150으로 늘렸습니다.

hidden_size를 높였을 때보다 layer 개수를 늘렸을 때 차이가 더 컸으며,
hidden_size는 layer가 1, 2 각각의 경우에 대하여 100, 150, 200로 변화시켜 accuracy를 출력한 결과,
layer=2, hidden_size=150일 때 가장 정확도가 높은 것을 확인할 수 있었습니다.

In [54]:
from google.colab import drive
drive.mount("/gdrive", force_remount=True)

Mounted at /gdrive


In [66]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import (DataLoader, TensorDataset)
from sklearn.metrics import accuracy_score

class BeforeSpacingRNN(nn.Module):

    def __init__(self, config): 
        super(BeforeSpacingRNN, self).__init__()

        self.eumjeol_vocab_size = config["eumjeol_vocab_size"]
        self.embedding_size = config["embedding_size"]
        self.hidden_size = config["hidden_size"]

        self.number_of_labels = config["number_of_labels"] 
        self.embedding = nn.Embedding(num_embeddings=self.eumjeol_vocab_size, embedding_dim=self.embedding_size, padding_idx=0)

        self.dropout = nn.Dropout(config["dropout"]) 
        
        # RNN layer
        self.bi_lstm = nn.LSTM(input_size=self.embedding_size, hidden_size=self.hidden_size, num_layers=1, batch_first=True, bidirectional=True)
        self.linear = nn.Linear(in_features=self.hidden_size*2, out_features=self.number_of_labels)
    
    def forward(self, inputs):
        eumjeol_inputs = self.embedding(inputs)
        hidden_outputs, hidden_states = self.bi_lstm(eumjeol_inputs)

        hidden_outputs = self.dropout(hidden_outputs)

        hypothesis = self.linear(hidden_outputs)

        return hypothesis


class SpacingRNN(nn.Module):

    def __init__(self, config): 
        super(SpacingRNN, self).__init__()

        self.eumjeol_vocab_size = config["eumjeol_vocab_size"]
        self.embedding_size = config["embedding_size"]
        self.hidden_size = config["hidden_size"]
        self.number_of_labels = config["number_of_labels"]
        self.embedding = nn.Embedding(num_embeddings=self.eumjeol_vocab_size, embedding_dim=self.embedding_size, padding_idx=0)
        self.dropout = nn.Dropout(config["dropout"])

        # RNN layer
        self.bi_lstm = nn.LSTM(input_size=self.embedding_size, hidden_size=self.hidden_size, num_layers=2, batch_first=True, bidirectional=True) # layer를 하나 늘림 -> num_layer=2
        self.linear = nn.Linear(in_features=self.hidden_size*2, out_features=self.number_of_labels)


    def forward(self, inputs):
        eumjeol_inputs = self.embedding(inputs)
        hidden_outputs, hidden_states = self.bi_lstm(eumjeol_inputs)
        hidden_outputs = self.dropout(hidden_outputs)
        hypothesis = self.linear(hidden_outputs)

        return hypothesis

In [67]:
def read_datas(file_path):
    with open(file_path, "r", encoding="utf8") as inFile:
        lines = inFile.readlines()

    datas = []

    for line in lines:
        pieces = line.strip().split("\t")
        eumjeol_sequence, label_sequence = pieces[0].split(), pieces[1].split()
        datas.append((eumjeol_sequence, label_sequence))

    return datas

def read_vocab_data(eumjeol_vocab_data_path):
    label2idx, idx2label = {"<PAD>":0, "B":1, "I":2}, {0:"<PAD>", 1:"B", 2:"I"}
    eumjeol2idx, idx2eumjeol = {}, {}

    with open(eumjeol_vocab_data_path, "r", encoding="utf8") as inFile:
        lines = inFile.readlines()

    for line in lines:
        eumjeol = line.strip()
        eumjeol2idx[eumjeol] = len(eumjeol2idx)
        idx2eumjeol[eumjeol2idx[eumjeol]] = eumjeol

    return eumjeol2idx, idx2eumjeol, label2idx, idx2label

def load_dataset(config):
    datas = read_datas(config["input_data"])
    eumjeol2idx, idx2eumjeol, label2idx, idx2label = read_vocab_data(config["eumjeol_vocab"])

    eumjeol_features, eumjeol_feature_lengths, label_features = [], [], []

    for eumjeol_sequence, label_sequence in datas:
        eumjeol_feature = [eumjeol2idx[eumjeol] for eumjeol in eumjeol_sequence]
        label_feature = [label2idx[label] for label in label_sequence]

        eumjeol_feature_length = len(eumjeol_feature)

        eumjeol_feature += [0] * (config["max_length"] - eumjeol_feature_length)
        label_feature  += [0] * (config["max_length"] - eumjeol_feature_length)

        eumjeol_features.append(eumjeol_feature)
        eumjeol_feature_lengths.append(eumjeol_feature_length)
        label_features.append(label_feature)

    eumjeol_features = torch.tensor(eumjeol_features, dtype=torch.long)
    eumjeol_feature_lengths = torch.tensor(eumjeol_feature_lengths, dtype=torch.long)
    label_features = torch.tensor(label_features, dtype=torch.long)

    return eumjeol_features, eumjeol_feature_lengths, label_features, eumjeol2idx, idx2eumjeol, label2idx, idx2label

In [68]:
def train(config):
    model = SpacingRNN(config).cuda()

    eumjeol_features, eumjeol_feature_lengths, label_features, eumjeol2idx, idx2eumjeol, label2idx, idx2label = load_dataset(config)

    train_features = TensorDataset(eumjeol_features, eumjeol_feature_lengths, label_features)
    train_dataloader = DataLoader(train_features, shuffle=True, batch_size=config["batch_size"])

    loss_func = nn.CrossEntropyLoss()

    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(config["epoch"]):

        model.train()
        costs = []

        for step, batch in enumerate(train_dataloader):
            optimizer.zero_grad()

            batch = tuple(t.cuda() for t in batch)

            inputs, input_lengths, labels = batch[0], batch[1], batch[2]

            hypothesis = model(inputs)

            cost = loss_func(hypothesis.reshape(-1, len(label2idx)), labels.flatten())

            cost.backward()
            optimizer.step()

            costs.append(cost.data.item())

        torch.save(model.state_dict(), os.path.join(output_dir, "epoch_{0:d}.pt".format(epoch + 1)))

        print("Improved Average cost : {}".format(np.mean(costs)))

In [69]:
def make_sentence(inputs, predicts, labels, idx2eumjeol, idx2label):

    predict_sentence, correct_sentence = "", ""

    for index in range(len(inputs)):
        eumjeol = idx2eumjeol[inputs[index]]
        correct_label = idx2label[labels[index]]
        predict_label = idx2label[predicts[index]]

        if (index == 0):
            predict_sentence += eumjeol
            correct_sentence += eumjeol
            continue

        if (predict_label == "B"):
            predict_sentence += " "
        predict_sentence += eumjeol

        if (correct_label == "B"):
            correct_sentence += " "
        correct_sentence += eumjeol

    return predict_sentence, correct_sentence

def tensor2list(input_tensor):
    return input_tensor.cpu().detach().numpy().tolist()

def test(config):
    eumjeol_features, eumjeol_feature_lengths, label_features, eumjeol2idx, idx2eumjeol, label2idx, idx2label = load_dataset(config)

    test_features = TensorDataset(eumjeol_features, eumjeol_feature_lengths, label_features)
    test_dataloader = DataLoader(test_features, shuffle=False, batch_size=1)

    model = SpacingRNN(config).cuda()
    
    model.load_state_dict(torch.load(os.path.join(config["output_dir_path"], config["model_name"])))

    total_hypothesis, total_labels = [], []

    for step, batch in enumerate(test_dataloader):

        model.eval()
        batch = tuple(t.cuda() for t in batch)

        inputs, input_lengths, labels = batch[0], batch[1], batch[2]

        hypothesis = model(inputs)

        hypothesis = torch.argmax(hypothesis, dim=-1)
        
        input_length = tensor2list(input_lengths[0])
        input = tensor2list(inputs[0])[:input_length]
        label = tensor2list(labels[0])[:input_length]
        hypothesis = tensor2list(hypothesis[0])[:input_length]

        total_hypothesis += hypothesis
        total_labels += label

        if (step < 10):
            predict_sentence, correct_sentence = make_sentence(input, hypothesis, label, idx2eumjeol, idx2label)
            print("정답 : " + correct_sentence)
            print("출력 : " + predict_sentence)
            print()

    print("Improved Accuracy : {}".format(accuracy_score(total_labels, total_hypothesis)))

In [80]:
def trainBefore(config):
    model = BeforeSpacingRNN(config).cuda()

    eumjeol_features, eumjeol_feature_lengths, label_features, eumjeol2idx, idx2eumjeol, label2idx, idx2label = load_dataset(config)

    train_features = TensorDataset(eumjeol_features, eumjeol_feature_lengths, label_features)
    train_dataloader = DataLoader(train_features, shuffle=True, batch_size=config["batch_size"])

    loss_func = nn.CrossEntropyLoss()

    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(config["epoch"]):

        model.train()
        costs = []

        for step, batch in enumerate(train_dataloader):
            optimizer.zero_grad()

            batch = tuple(t.cuda() for t in batch)

            inputs, input_lengths, labels = batch[0], batch[1], batch[2]

            hypothesis = model(inputs)

            cost = loss_func(hypothesis.reshape(-1, len(label2idx)), labels.flatten())

            cost.backward()
            optimizer.step()

            costs.append(cost.data.item())

        torch.save(model.state_dict(), os.path.join(output_dir_before, "epoch_{0:d}.pt".format(epoch + 1)))

        print("Before Average cost : {}".format(np.mean(costs)))

def testBefore(config):
    eumjeol_features, eumjeol_feature_lengths, label_features, eumjeol2idx, idx2eumjeol, label2idx, idx2label = load_dataset(config)

    test_features = TensorDataset(eumjeol_features, eumjeol_feature_lengths, label_features)
    test_dataloader = DataLoader(test_features, shuffle=False, batch_size=1)

    model = BeforeSpacingRNN(config).cuda()
    
    model.load_state_dict(torch.load(os.path.join(config["output_dir_path"], config["model_name"])))

    total_hypothesis, total_labels = [], []

    for step, batch in enumerate(test_dataloader):

        model.eval()
        batch = tuple(t.cuda() for t in batch)

        inputs, input_lengths, labels = batch[0], batch[1], batch[2]

        hypothesis = model(inputs)

        hypothesis = torch.argmax(hypothesis, dim=-1)
        input_length = tensor2list(input_lengths[0])
        input = tensor2list(inputs[0])[:input_length]
        label = tensor2list(labels[0])[:input_length]
        hypothesis = tensor2list(hypothesis[0])[:input_length]

        total_hypothesis += hypothesis
        total_labels += label

        if (step < 10):
            predict_sentence, correct_sentence = make_sentence(input, hypothesis, label, idx2eumjeol, idx2label)
            print("정답 : " + correct_sentence)
            print("출력 : " + predict_sentence)
            print()

    print("Before Accuracy : {}".format(accuracy_score(total_labels, total_hypothesis)))

In [87]:
if(__name__=="__main__"):
    root_dir = "/gdrive/My Drive/ml_colab/week12"
    output_dir = os.path.join(root_dir, "output")
    output_dir_before = os.path.join(root_dir, "outputbefore")

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    if not os.path.exists(output_dir_before):
        os.makedirs(output_dir_before)
        


    # 개선 전 모델에 대한 config
    configBefore = {"mode": "test",
              "model_name":"epoch_{0:d}.pt".format(5),
              "input_data":os.path.join(root_dir, "test.txt"),
              "output_dir_path":output_dir_before,
              "eumjeol_vocab": os.path.join(root_dir, "eumjeol_vocab.txt"),
              "label_vocab": os.path.join(root_dir, "label_vocab.txt"),
              "eumjeol_vocab_size": 2458,
              "embedding_size": 100,
              "hidden_size": 100,
              "max_length": 920,
              "number_of_labels": 3,
              "epoch":5,
              "batch_size":64,
              "dropout":0.3
              }

    # 개선 후 모델에 대한 config
    config = {"mode": "test",
              "model_name":"epoch_{0:d}.pt".format(5),
              "input_data":os.path.join(root_dir, "test.txt"),
              "output_dir_path":output_dir,
              "eumjeol_vocab": os.path.join(root_dir, "eumjeol_vocab.txt"),
              "label_vocab": os.path.join(root_dir, "label_vocab.txt"),
              "eumjeol_vocab_size": 2458,
              "embedding_size": 100,
              "hidden_size": 150, # 원래: 100 -> 150
              # 100, 150, 200 으로 수정하며 결과 확인 결과, 150일 때 accuracy가 가장 높은 수치를 보임을 확인했음.
              "max_length": 920,
              "number_of_labels": 3,
              "epoch":5,
              "batch_size":64,
              "dropout":0.3
              }

    if(config["mode"] == "train"):
        print("[개선 전]")
        print()
        trainBefore(configBefore)
        print()
        print()
        print("[개선 후]")
        print()
        train(config)
    else:
        print("[개선 전]")
        print()
        testBefore(configBefore)
        print()
        print()
        print("[개선 후]")
        print()
        test(config)

[개선 전]

정답 : 그러고 보니 경리는 윤보혜의 근황에 대해 아는 것이 없어보였다.
출력 : 그러고 보니 경리는 윤보혜의 근황에 대해 아는 것이 없어보였다.

정답 : 이제 7년 대환란이 눈앞에 닥쳐왔습니다.
출력 : 이 제7년 대환란 이 눈앞에 닥쳐왔습니다.

정답 : 이 관찰자에게 장치(예를 들면 90의 경사각을 가진 두 거울)를 제공하여, 같은 시각에 A와 B 두 곳을 한꺼번에 관찰할 수 있게 한다.
출력 : 이 관찰자에게 장치(예를 들면 90의 경사각을 가진 두거울)를 제공하여, 같은 시각에 A와 B두곳을 한 꺼번에 관찰할 수 있게 한다.

정답 : "먼저 약속을 어긴 쪽은 한달준 그 놈이었어."
출력 : "먼저약속을 어긴 쪽은 한 달준 그 놈이었어."

정답 : 레이첼.
출력 : 레이첼.

정답 : 처남은 의사의 진단서를 북북 찢어버렸다.
출력 : 처남은 의 사의 진단서를 북북찢어버렸다.

정답 : 그 전화를 받았던 날, 과일을 벗기던 혜숙이가 물었었다.
출력 : 그 전화를 받았던 날, 과 일을 벗기던 혜숙이가 물었었다.

정답 : "저도 처음에는 금변호사님처럼 그렇게 생각했지요. 그러나 범인은 경계선을 사이에 두고 한쪽 발은 테이블쪽에 두고 한쪽 발을 홀 중앙에 두고 있던 있던 게 분명해요."
출력 : "저도 처음에는 금변호사님처럼 그렇게 생각했지요. 그러나 범인은 경계선을 사이에 두고 한 쪽발은 테이 블쪽에 두고 한 쪽발을 홀중앙에 두고 있던 있던게 분명해요."

정답 : 1979년 <모두를 위한 정의>로 마지막 오스카 주연상 후보에 올랐었으나 이 영화를 제외하고는 70년대 말과 80년대 초를 통틀어 별로 신통한 영화에는 나오지 않았다.
출력 : 1979년 <모두를 위한 정의 >로 마지 막 오스카주연상후보에 올랐었으나이 영화를 제외하고는 70년대말과 80년대초를 통틀어별로 신통한 영화에는 나오지 않았다.

정답 : 경찰은 처음에 그 사건이 아그자 혼자 저지른 단독 범행인 줄 알았지만 조사 결과 두 명 이상의 공범이 있었음이 밝혀졌고, 수사가 진